In [1]:
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('../Style.mplstyle')
import pandas as pd
import os
import sys
import scipy
import matplotlib.dates as mdates
from datetime import datetime
sys.path.append('..')
from read_data_functions import *
from plot_functions import *
from calculations import *
import warnings
warnings.filterwarnings('ignore')

In [2]:
# Define paths to data
parent_path = '../../../../'
path = 'L:/PG-Nanoteknologi/PROJEKTER/2024 Laura og Nan/Lund/'

# Read data
NS_SMPS = read_SMPS(path, parent_path, [0, 0])
OPS_APS = read_OPS(path, parent_path, 0)
CPC = read_CPC(f'{path}CPC/', parent_path)

# Define dictionary keys relavent for each experiment
CPC_dict_keys = ['test7_CPC', 'test8_CPC', 'test9_CPC', 'test10_CPC']
NS_dict_keys = ['2024-10-16_NanoScan', '2024-10-16_NanoScan', '2024-10-17_NanoScan', '2024-10-17_NanoScan']
dict_keys = ['Exp1', 'Exp2', 'Exp3', 'Exp4']
APS_dict_keys = ['20241016_APS', '20241016_APS_2_exp', '20241017_APS', '20241017_APS']
SMPS_dict_keys = ['20241016_SMPS', '20241016_SMPS_2_exp', '20241017_SMPS', '20241017_SMPS']

# Define relevant df keys for each instrument
NS_bins = NS_SMPS['2024-10-21_NanoScan'].keys()[3:16].to_list()
OPS_bins = OPS_APS['Exp1'].keys()[1:17].to_list()
OPS_mid_point = [0.337, 0.419, 0.522, 0.650, 0.809, 1.007, 1.254, 1.562, 1.944, 2.421, 3.014, 3.752, 4.672, 5.816, 7.241, 9.015]
APS_bins = OPS_APS['20241016_APS'].keys()[1:53].to_list()
SMPS_bins = NS_SMPS['20241016_SMPS'].keys()[4:117].to_list()

# Subtract HEPA filter measurements from NanoScan data
NS_bg_mean, NS_bg_std, NS_bg_error = bin_mean(['2024/10/21 11:45:00', '2024/10/21 14:10:00'], 
                                              NS_SMPS['2024-10-21_NanoScan'], NS_bins, 'Time', 0.1)
for key in NS_dict_keys:
    df = NS_SMPS[key]
    for i, bin in enumerate(NS_bins):
        df[bin] = df[bin] - NS_bg_mean[i]
    NS_SMPS[key] = df

# Change OPS df keys from bin cut-point to bin mid-point
for dict_key in dict_keys:
    for old_key, new_key in zip(OPS_bins, OPS_mid_point):
        OPS_APS[dict_key] = OPS_APS[dict_key].rename(columns = {old_key: new_key})

# Convert from dN/dlogDp to N for APS
APS_bin_mean = [(0.486968 + 0.523)/2]
for key in OPS_APS['20241016_APS'].keys()[2:53]:
    APS_bin_mean.append(float(key))
APS_cut_point = np.concatenate(([0.486968], (np.array(APS_bin_mean)[1:]+np.array(APS_bin_mean)[:-1])/2, [20.53]))

for key in APS_dict_keys[1:3]:
    for i, bin in enumerate(APS_bins):
        bin_width = np.log10(APS_cut_point[i+1]) - np.log10(APS_cut_point[i])
        OPS_APS[key][bin] = pd.to_numeric(OPS_APS[key][bin]) * bin_width

# Convert from mass to number for APS data from first experiment
APS_number = pd.DataFrame({'Time': OPS_APS['20241016_APS']['Time']})
for i, key in enumerate(APS_bins):
    OPS_APS['20241016_APS'][key] = pd.to_numeric(OPS_APS['20241016_APS'][key], errors='coerce') * 1.2 # Correct density
    APS_number[key] = OPS_APS['20241016_APS'][key] / ((1.2 / 10**6) * (np.pi / 6) * APS_bin_mean[i]**3 * 10**6) # in #/cm**3
OPS_APS['20241016_APS'] = APS_number

# Convert from dN/dlogDp to N for SMPS
SMPS_bin_mean = []
for key in NS_SMPS['20241016_SMPS'].keys()[4:117]:
    SMPS_bin_mean.append(float(key) / 1000)
SMPS_cut_point = np.concatenate(([0.017], (np.array(SMPS_bin_mean)[1:]+np.array(SMPS_bin_mean)[:-1])/2, [1.0]))

for key in SMPS_dict_keys[:3]:
    for i, bin in enumerate(SMPS_bins):
        bin_width = np.log10(SMPS_cut_point[i+1]) - np.log10(SMPS_cut_point[i])
        NS_SMPS[key][bin] = pd.to_numeric(NS_SMPS[key][bin]) * bin_width

Failed to read file with separation: 	
Failed to read file with separation: ,
Failed to read file with separation: ,


In [3]:
def remove_bg(data_dict, dict_keys, new_keys, df_keys, bg_timestamps):
    new_dict = {}

    for i, key in enumerate(dict_keys):
        df = data_dict[key]
        new_df = pd.DataFrame({'Time': df['Time']})

        if len(df_keys) <= 1:
            bg_time, bg_conc = time_filtered_arrays(df, None, bg_timestamps[i], df_keys[0])
            new_df[df_keys[0]] = df[df_keys[0]] - bg_conc.mean()
            new_df[df_keys[0]][new_df[df_keys[0]] < 0] = 0

        else:
            background, std, e = bin_mean(bg_timestamps[i], df, df_keys, 'Time', None)

            for bin, bg in zip(df_keys, background):
                new_df[bin] = pd.to_numeric(df[bin], errors='coerce') - bg
                new_df[bin][new_df[bin] < 0] = 0
        
        if new_keys != None:
            new_dict[new_keys[i]] = new_df
        else:
            new_dict[key] = new_df

    return new_dict

In [4]:
timestamps_BG = [['2024-10-16 10:17:00', '2024-10-16 10:26:00'],
                 ['2024-10-16 14:43:00', '2024-10-16 14:53:00'],
                 ['2024-10-17 09:48:00', '2024-10-17 09:59:00'],
                 ['2024-10-17 14:10:00', '2024-10-17 14:20:00']]

OPS = remove_bg(OPS_APS, dict_keys, None, OPS_mid_point, timestamps_BG)
APS = remove_bg(OPS_APS, APS_dict_keys, dict_keys, APS_bins, timestamps_BG)
NS = remove_bg(NS_SMPS, NS_dict_keys, dict_keys, NS_bins, timestamps_BG)
SMPS = remove_bg(NS_SMPS, SMPS_dict_keys, dict_keys, SMPS_bins, timestamps_BG)
CPC = remove_bg(CPC, CPC_dict_keys, dict_keys, [CPC['test7_CPC'].keys()[1]], timestamps_BG)

In [5]:
timestamps_exp = [['2024-10-16 10:26:00', '2024-10-16 12:26:00'],
                  ['2024-10-16 14:54:00', '2024-10-16 15:34:00'],
                  ['2024-10-17 10:00:00', '2024-10-17 12:00:00'],
                  ['2024-10-17 14:17:00', '2024-10-17 14:57:00']]

In [6]:
# Total concentrations
for i, key in enumerate(dict_keys):
    print(f'Experiment {i+1}:')
    start, end = pd.to_datetime(timestamps_exp[i][0]), pd.to_datetime(timestamps_exp[i][1])

    # NanoScan
    time_NS = pd.to_datetime(NS[key]['Time'])
    time_filter_NS = (time_NS >= start) & (time_NS <= end)
    df_NS = NS[key][time_filter_NS]

    PN01_array_NS = df_NS.iloc[:,1:10].sum(axis = 1)
    PN03_array_NS = df_NS.iloc[:,1:13].sum(axis = 1)
    PN01_NS = PN01_array_NS.sum()
    PN03_NS = PN03_array_NS.sum()
    print(f'NanoScan:   PN0.1 = {PN01_NS:.2f} and PN0.3 = {PN03_NS:.2f}')

    # SMPS
    time_SMPS = pd.to_datetime(SMPS[key]['Time'])
    time_filter_SMPS = (time_SMPS >= start) & (time_SMPS <= end)
    df_SMPS = SMPS[key][time_filter_SMPS]

    PN01_array_SMPS = df_SMPS.iloc[:,1:51].sum(axis = 1)
    PN03_array_SMPS = df_SMPS.iloc[:,1:81].sum(axis = 1)
    PN1_array_SMPS = df_SMPS.iloc[:,1:].sum(axis = 1)
    PN01_SMPS = PN01_array_SMPS.sum()
    PN03_SMPS = PN03_array_SMPS.sum()
    PN1_SMPS = PN1_array_SMPS.sum()
    print(f'SMPS:   PN0.1 = {PN01_SMPS:.2f} and PN0.3 = {PN03_SMPS:.2f} and PN1 = {PN1_SMPS:.2f}')

    # CPC
    time_CPC = pd.to_datetime(CPC[key]['Time'])
    time_filter_CPC = (time_CPC >= start) & (time_CPC <= end)
    df_CPC = CPC[key][time_filter_CPC]

    PN1_CPC = df_CPC[CPC[key].keys()[1]].sum()
    print(f'CPC: PN1 = {PN1_CPC:.2f}')

    # OPS
    time_OPS = pd.to_datetime(OPS[key]['Time'])
    time_filter_OPS = (time_OPS >= start) & (time_OPS <= end)
    df_OPS = OPS[key][time_filter_OPS]

    PN1_array_OPS = df_OPS.iloc[:,1:7].sum(axis = 1)
    PN25_array_OPS = df_OPS.iloc[:,1:11].sum(axis = 1)
    PN10_array_OPS = df_OPS.iloc[:,1:].sum(axis = 1)
    PN1_OPS = PN1_array_OPS.sum()
    PN25_OPS = PN25_array_OPS.sum()
    PN10_OPS = PN10_array_OPS.sum()
    print(f'OPS:   PN1 = {PN1_OPS:.2f} and PN2.5 = {PN25_OPS:.2f} and PN10 = {PN10_OPS:.2f}')

    # APS
    time_APS = pd.to_datetime(APS[key]['Time'])
    time_filter_APS = (time_APS >= start) & (time_APS <= end)
    df_APS = APS[key][time_filter_APS]

    PN25_array_APS = df_APS.iloc[:,1:24].sum(axis = 1)
    PN10_array_APS = df_APS.iloc[:,1:43].sum(axis = 1)
    PN20_array_APS = df_APS.iloc[:,1:].sum(axis = 1)
    PN25_APS = PN25_array_APS.sum()
    PN10_APS = PN10_array_APS.sum()
    PN20_APS = PN20_array_APS.sum()
    print(f'APS:   PN2.5 = {PN25_APS:.2f} and PN10 = {PN10_APS:.2f} and PN20 = {PN20_APS:.2f}')


Experiment 1:
NanoScan:   PN0.1 = 20039.51 and PN0.3 = 33113.51
SMPS:   PN0.1 = 77.96 and PN0.3 = 273.80 and PN1 = 726.91
CPC: PN1 = 232532.84
OPS:   PN1 = 3061.10 and PN2.5 = 3418.58 and PN10 = 3461.83
APS:   PN2.5 = 4.48 and PN10 = 5.11 and PN20 = 5.12
Experiment 2:
NanoScan:   PN0.1 = 39589.17 and PN0.3 = 57668.74
SMPS:   PN0.1 = 2919.27 and PN0.3 = 4190.26 and PN1 = 4754.30
CPC: PN1 = 1823043.52
OPS:   PN1 = 4437.51 and PN2.5 = 4925.70 and PN10 = 4977.77
APS:   PN2.5 = 7193.40 and PN10 = 7954.90 and PN20 = 7955.20
Experiment 3:
NanoScan:   PN0.1 = 3880.03 and PN0.3 = 6628.93
SMPS:   PN0.1 = 56.87 and PN0.3 = 163.95 and PN1 = 390.27
CPC: PN1 = 150240.18
OPS:   PN1 = 1866.25 and PN2.5 = 2058.62 and PN10 = 2075.04
APS:   PN2.5 = 3057.63 and PN10 = 3405.53 and PN20 = 3405.69
Experiment 4:
NanoScan:   PN0.1 = 8048.08 and PN0.3 = 13083.03
SMPS:   PN0.1 = 97.15 and PN0.3 = 207.69 and PN1 = 603.41
CPC: PN1 = 291537.80
OPS:   PN1 = 3605.13 and PN2.5 = 4069.82 and PN10 = 4101.96
APS:   PN2.5